## Automated anisotropic resistivity inversion for efficient formation evaluation and uncertainty quantification

### Misael M. Morales, Ali Eghbali, Oriyomi Raheem, Michael Pyrcz, Carlos Torres-Verdin
***
## PINN-based Inversion (PyTorch)
***

In [1]:
from main import *

check_torch()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
case1, case2, synthetic1, synthetic2 = load_all_data()


------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.2.2+cu121 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3080
------------------------------------------------------------

Name              : Source                : Shape
----------------- : --------------------- : -----------
Field Case 1      : (Chevron)             : (2399, 12)
Field Case 2      : (AkerBP)              : (11143, 12)
Synthetic Case 1  : (Laminated)           : (801, 14)
Synthetic Case 2  : (Laminated+Dispersed) : (415, 10)


***
### Physics-informed neural network inversion

In [2]:
class ResInvLoss(nn.Module):
    def __init__(self, ddmax=100, lambda_reg=1e-5, lambda_p=2):
        super(ResInvLoss, self).__init__()
        self.lambda_reg = lambda_reg
        self.lambda_p   = lambda_p
        self.ddmax      = ddmax

    def forward(self, inputs, outputs):
        Rv_true = inputs[:, 0]
        Rh_true = inputs[:, 1]
        dd_true = inputs[:, 2]/self.ddmax
        Rvsh    = inputs[:, 3]
        Rhsh    = inputs[:, 4]

        Csh_pred = outputs[:, 0]
        Rss_pred = outputs[:, 1]

        eq1 = (Csh_pred*Rvsh + (1-Csh_pred)*Rss_pred) - (Rv_true)
        eq2 = 1/(Csh_pred/Rhsh + (1-Csh_pred)/Rss_pred) - (Rh_true)
        eqs = torch.stack([eq1, eq2], dim=-1)

        wd1, wd2 = 1/Rv_true/dd_true, 1*Rh_true/dd_true
        Wdm = torch.stack([wd1, wd2], dim=-1)

        costf = torch.norm(torch.matmul(Wdm.T, eqs), p=2)
        regPa = self.lambda_reg*torch.norm(outputs, p=self.lambda_p)

        return  costf + regPa

In [3]:
class ResInvPINN(nn.Module):
    def __init__(self, hidden_dim:int=128):
        super(ResInvPINN, self).__init__()
        self.fc1 = nn.Linear(2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 2)

    def constraints(self, d):
        c, s = d[:, 0], d[:, 1]
        c = nn.Sigmoid()(c)
        return torch.stack([c, s], dim=-1)
       
    def forward(self, x):
        x = x[:, :2]
        
        x = self.fc1(x)
        x = nn.Tanh()(x)

        x = self.fc2(x)
        x = nn.Tanhshrink()(x)

        x = self.fc3(x)
        x = self.constraints(x)

        return x

In [ ]:
column_names = ['CALI', 'AT10', 'AT30', 'AT60', 'AT90', 'GR', 'Rv', 'Rh', 'Rvsh', 'Rhsh']
dd = 'GR'

zstart = int(np.argwhere(case1.index==9720).squeeze())
zend   = int(np.argwhere(case1.index==10110).squeeze())
data1  = case1.iloc[zstart:zend]

zstart = int(np.argwhere(case2.index==6292.75).squeeze())
zend   = int(np.argwhere(case2.index==9078.25).squeeze())
data2  = case2.iloc[zstart:zend]

data3 = synthetic1.dropna()
data4 = synthetic2.dropna()
if dd not in data4.columns:
    print('Selected normalizing data not in columns of Well 4, using "GR" instead.')
    data4.loc[:,dd] = synthetic2['GR']

data_all = pd.concat([data2, data1, data3, data4], ignore_index=False)

res_aniso = data_all[['Rv', 'Rh', dd, 'Rvsh', 'Rhsh', 'WIDX']]
inputs    = torch.tensor(res_aniso.values, dtype=torch.float32).to(device)
print('Inputs: {}'.format(inputs.shape))

dataset        = TensorDataset(inputs)
train_percent  = 0.85
xtrain, xtest  = random_split(dataset, [int(train_percent*len(dataset)), len(dataset)-int(train_percent*len(dataset))])
xtrain, xvalid = random_split(xtrain, [int(train_percent*len(xtrain)), len(xtrain)-int(train_percent*len(xtrain))])
print('X_train: {} | X_valid: {} | X_test: {}'.format(len(xtrain), len(xvalid), len(xtest)))

batch_size  = 32
trainloader = DataLoader(xtrain, batch_size=batch_size, shuffle=True)
validloader = DataLoader(xvalid, batch_size=batch_size, shuffle=True)

model     = ResInvPINN(hidden_dim=150).to(device)
criterion = ResInvLoss(ddmax=data1[dd].max(), lambda_reg=1e-10).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
print('# of Parameters: {:,}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

In [ ]:
epochs, monitor = 501, 100
train_loss, valid_loss = [], []
for epoch in range(epochs):
    # training
    epoch_train_loss = []
    model.train()
    for i, x in enumerate(trainloader):
        optimizer.zero_grad()
        y = model(x[0])
        loss = criterion(x[0], y)
        loss.backward()
        optimizer.step()
        epoch_train_loss.append(loss.item())
    train_loss.append(np.mean(epoch_train_loss))
    # validation
    model.eval()
    epoch_valid_loss = []
    with torch.no_grad():
        for x in validloader:
            y = model(x[0])
            loss = criterion(x[0], y)
            epoch_valid_loss.append(loss.item())
        valid_loss.append(np.mean(epoch_valid_loss))
    # progress
    if epoch % monitor == 0:
        print('Epoch: {} | Loss: {:.4f} | Valid Loss: {:.4f}'.format(epoch, train_loss[-1], valid_loss[-1]))
losses = (train_loss, valid_loss)
plot_loss(losses)

In [ ]:
y_pred = model(inputs[:,:2]).cpu().detach().numpy().squeeze()
Csh_pred, Rss_pred = [y_pred[:, i] for i in range(y_pred.shape[1])]
print('Csh: min={:.3f} | max={:.3f}'.format(Csh_pred.min(), Csh_pred.max()))

Rv_true = res_aniso['Rv'].values
Rh_true = res_aniso['Rh'].values
Rvsh    = res_aniso['Rvsh'].values
Rhsh    = res_aniso['Rhsh'].values

Rv_sim = (Csh_pred*Rvsh + (1-Csh_pred)*Rss_pred)
Rh_sim = 1/(Csh_pred/Rhsh + (1-Csh_pred)/Rss_pred)

Rv_err = np.abs((Rv_sim - Rv_true)/Rv_true) * 100
Rh_err = np.abs((Rh_sim - Rh_true)/Rh_true) * 100

pinn_sol = pd.DataFrame({'Csh_pred':Csh_pred, 'Rss_pred':Rss_pred, 
                         'Rv_sim':Rv_sim, 'Rh_sim':Rh_sim,
                         'Rv_err':Rv_err, 'Rh_err':Rh_err}, 
                         index=res_aniso.index)

results = pd.concat([data_all, pinn_sol], axis=1)
results.to_csv('results/pinn_solution.csv', index=True)

error_metrics(results)

In [ ]:
s = 'Chevron 2009'
plot_pinn_results(results[data_all['WIDX']==1], suptitle=s)
gradientbased_results = pd.read_csv('results/gradient_based_solution_Chevron.csv', index_col=0)
plot_pinn_gb_comparison(results[data_all['WIDX']==1], gradientbased_results, suptitle=s)

In [ ]:
s = 'AkerBP Hanz Prospect'
plot_pinn_results(results[data_all['WIDX']==2], suptitle=s)
gradientbased_results = pd.read_csv('results/gradient_based_solution_AkerBP.csv', index_col=0)
plot_pinn_gb_comparison(results[data_all['WIDX']==2], gradientbased_results, suptitle=s)

In [ ]:
s = 'Synthetic Case 1 (Laminated)'
plot_pinn_results(results[data_all['WIDX']==3], figsize=(12,12), suptitle=s)
gradientbased_results = pd.read_csv('results/gradient_based_solution_synthetic1.csv', index_col=0).iloc[22:]
plot_pinn_gb_comparison(results[data_all['WIDX']==3], gradientbased_results, suptitle=s)

In [ ]:
s = 'Synthetic Case 2 (Laminated + Dispersed)'
plot_pinn_results(results[data_all['WIDX']==4], suptitle=s)
gradientbased_results = pd.read_csv('results/gradient_based_solution_synthetic2.csv', index_col=0)
plot_pinn_gb_comparison(results[data_all['WIDX']==4], gradientbased_results, suptitle=s)

***
## Uncertainty Quantification

In [ ]:
s1 = results[data_all['WIDX']==3]
s2 = results[data_all['WIDX']==4]
synthetic_all = pd.concat([s1, s2], ignore_index=False)
s_split = len(s1)
print('Synthetic 1: {} | Synthetic 2: {}'.format(s1.shape, s2.shape))
print('Synthetic All: {}'.format(synthetic_all.shape))

n_realizations = 1000
y_syn_preds = np.zeros((n_realizations, synthetic_all.shape[0], 2))
for i in range(n_realizations):
    noisy_inputs = inputs + torch.randn_like(d)*0.1
    y_syn_preds[i] = model(noisy_inputs).cpu().detach().numpy().squeeze()
print(y_syn_preds.shape)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(12.5,6.5))
ax1, ax2, ax3, ax4 = axs
titles = ['Chevron2009', 'AkerBP', 'Synthetic 1', 'Synthetic 2']

for i in range(50):
    Csh_pred, Rss_pred = y_preds[i, :, 0], y_preds[i, :, 1]
    Rvsh, Rhsh = results['Rvsh'].values, results['Rhsh'].values
    Rvsim = Csh_pred*Rvsh + (1-Csh_pred)*Rss_pred
    Rhsim = 1/(Csh_pred/Rhsh + (1-Csh_pred)/Rss_pred)
    ax1.plot(Rvsim[split:], r1.index, c='rosybrown', alpha=0.1)
    ax1.plot(Rhsim[split:], r1.index, c='lightsteelblue', alpha=0.1)
    ax2.plot(Rvsim[:split], r2.index, c='rosybrown', alpha=0.1)
    ax2.plot(Rhsim[:split], r2.index, c='lightsteelblue', alpha=0.1)

    c, r = y_syn_preds[i,:,0], y_syn_preds[i,:,1]
    rvsh, rhsh = synthetic_all['Rvsh'].values, synthetic_all['Rhsh'].values
    Rvsim = c*rvsh + (1-c)*r
    Rhsim = 1/(c/rhsh + (1-c)/r)
    ax3.plot(Rvsim[:s_split], s1.index, c='rosybrown', alpha=0.1)
    ax3.plot(Rhsim[:s_split], s1.index, c='lightsteelblue', alpha=0.1)
    ax4.plot(Rvsim[s_split:], s2.index, c='rosybrown', alpha=0.1)
    ax4.plot(Rhsim[s_split:], s2.index, c='lightsteelblue', alpha=0.1)

ax1.plot(r1['Rv'], r1.index, c='darkred', ls='--', label='True Rv')
ax1.plot(r1['Rh'], r1.index, c='darkblue', ls='--', label='True Rh')

ax2.plot(r2['Rv'], r2.index, c='darkred', ls='--', label='True Rv')
ax2.plot(r2['Rh'], r2.index, c='darkblue', ls='--', label='True Rh')

ax3.plot(s1['Rv'], s1.index, c='darkred', ls='--', label='True Rv')
ax3.plot(s1['Rh'], s1.index, c='darkblue', ls='--', label='True Rh')

ax4.plot(s2['Rv'], s2.index, c='darkred', ls='--', label='True Rv')
ax4.plot(s2['Rh'], s2.index, c='darkblue', ls='--', label='True Rh')

[ax.set_title(titles[i]) for i, ax in enumerate(axs)]
[ax.set_xscale('log') for ax in axs]
[ax.grid(True, which='both') for ax in axs]
[ax.legend(loc='lower left') for ax in axs]
ax1.invert_yaxis(); ax2.invert_yaxis()
plt.tight_layout()
plt.show()

***
# END